In [1]:
# Imports
import torch
import torchvision # torch package for vision related things
import torch.nn.functional as F  # Parameterless functions, like (some) activation functions
import torchvision.datasets as datasets  # Standard datasets
import torchvision.transforms as transforms  # Transformations we can perform on our dataset for augmentation
from torch import optim  # For optimizers like SGD, Adam, etc.
from torch import nn  # All neural network modules
from torch.utils.data import DataLoader  # Gives easier dataset managment by creating mini batches etc.
from tqdm import tqdm  # For nice progress bar!


In [17]:
class CNN(nn.Module):
  def __init__(self,in_channels=3,num_classes=10):
    super(CNN,self).__init__()
    self.conv1 = nn.Conv2d(
        in_channels=in_channels,
        out_channels=12,
        kernel_size=(3,3),
        stride=(1,1),
        padding=(1,1),
    )
    self.pool = nn.MaxPool2d(kernel_size=(2,2),stride=(2,2))
    self.conv2 = nn.Conv2d(
        in_channels=12,
        out_channels=24,
        kernel_size=(3,3),
        stride=(1,1),
        padding=(1,1),
    )
    self.conv3 = nn.Conv2d(
        in_channels=24,
        out_channels=36,
        kernel_size=(3,3),
        stride=(1,1),
        padding=(1,1),
    )
    self.fc1 = nn.Linear(36*4*4, num_classes)

  def forward(self,x):
    # print("x.shape0: ",x.shape)
    x= F.relu(self.conv1(x))
    x=self.pool(x)
    # print("x.shape1: ",x.shape)
    x=F.relu(self.conv2(x))
    x=self.pool(x)
    # print("x.shape2: ",x.shape)
    x=F.relu(self.conv3(x))
    x=self.pool(x)
    # print("x.shape3: ",x.shape)
    x=x.reshape(x.shape[0],-1)
    x=self.fc1(x)
    return x





#setDevice
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

#hyperParameters

in_channels=3
num_classes=10
learning_rate=0.001
batch_size=100
num_epochs=10


#LoadData

train_dataset=datasets.CIFAR10(root="datasets/",train=True,transform= transforms.ToTensor(),download=True)
test_dataset=datasets.CIFAR10(root="datasets/",train=False,transform= transforms.ToTensor(),download=True)
train_loader= DataLoader(dataset=train_dataset,batch_size=batch_size,shuffle=True)
test_loader= DataLoader(dataset=test_dataset,batch_size=batch_size,shuffle=True)

#initialize network

model = CNN(in_channels=in_channels,num_classes=num_classes).to(device)

#loss and optimizer

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(),lr=learning_rate)

#Training

for epoch in range(num_epochs):
  for batch_idx, (data,targets) in enumerate(tqdm(train_loader)):
    #get data to cuda if possible
    data = data.to(device)
    targets = targets.to(device)

    #forward

    scores=model(data)
    loss = criterion(scores,targets)

    #backward

    optimizer.zero_grad()
    loss.backward()

    #gradient descent or adam step
    optimizer.step()

#Check accuracy on training and test to see how good our model is

def check_accuracy(loader,model):
  num_correct = 0
  num_samples = 0
  model.eval()

  with torch.no_grad():
    for x,y in loader:
      x = x.to(device=device)
      y = y.to(device=device)

      scores = model(x)
      test, predictions = scores.max(1)
      num_correct += (predictions==y).sum()
      num_samples += predictions.size(0)


  model.train()
  return num_correct/num_samples

print(f"Accuracy on training set: {check_accuracy(train_loader, model)*100:.2f}")
print(f"Accuracy on test set: {check_accuracy(test_loader, model)*100:.2f}")


Files already downloaded and verified
Files already downloaded and verified


100%|██████████| 500/500 [00:07<00:00, 70.52it/s]


Accuracy on training set: 66.45
Accuracy on test set: 64.34
